The order of viewing this project could be:

•	Data processing. ipynb

•	Decision Point1.ipynb

•	Decision Point2.ipynb

•	Decision Point3.ipynb

The notebooks contain sufficient explanations intent and insights in the analysis.
Please reach out directly at shiva.sync@gmail.com for any issues.





## Assumptions and Considerations:

1)	I have restricted this Data Analysis to 2-bedroom housing properties in New York.

2)	The houses with room type “Private Room” in Airbnb data in a 2-bedroom setting apartment has a price for only 1 night’s stay in room. This has been multiplied by 2 for such properties as if rental management can potentially earn double rent of this property as it has 2 bedrooms. However, the overall rental price per night may not be exactly double as the other room may be smaller or have lesser or more appeal than the room in the listing.

3)	Occupancy for the Airbnb listings have been assumed to be 75% throughout the year

4)	The value of each property is assumed to increase/decrease every year by a fixed  rate.  The calculation of this rate has been discussed section 2.


## Metrics Created:

### Annual_Rent: 

This is annual rent collected from a property in Airbnb assuming 75% occupancy using rent price per night from Zillow data. This value is assumed to be fixed.

### Payback_Period: 

This is the number of years required to earn back the initial cost of investment from annual rental income from property assuming 75% occupancy throughout the year.


### Occupancy:

0.75*365 (Assumed occupancy of all properties in Airbnb)

### Rate: 
So, we all are aware that properties value appreciate/depreciate depending upon the neighborhood and locality. In the Zillow data we have the average housing prices for a zip code for as far back as 1996 but these columns have lot of missing values. However, we do have complete data from 06-2007. The difference between in prices in 06-2007 and 06-2017 divided by 10 has been taken as an average rate by which a property rises in that zip code. Although a naïve approach, it still captures the essence of appreciation/depreciation of property by zip codes over the years and would be useful in adding a location specific factor in Return on Investment calculations.

\begin{equation*}
\frac{Property Value in 2017 -  Property Value in 2007}{Property Value in 2007 * Difference in Years}
\end{equation*}







## Roi_Equity: 

This metric is return on investment in 10 years assuming rent appreciates at a constant rate without including the equity value of the property.


\begin{equation*}
\frac{Price of Property*(1+\frac{Rate}{100})^{10}- Current Price of the property}{Current Price of the property}
\end{equation*}










## Roi: 

This metric takes into account the return on investment through 
 equity appreciation i.e. the increase in the price of the house is also taken into account to calculate roi. This has been calculated for 10 years as well.

It is calculated as:


A=Annual Rent in 10 years

\begin{equation*}
Annual Rent*10
\end{equation*}




B= Increase in cost of property in 10 Years



\begin{equation*}
Price of Property*(1+\frac{Rate}{100})^{10}- Current Price of the property
\end{equation*}



Roi= (A+B)/Price of Property

\begin{equation*}
\frac{(A+B)}{Price Of Property}
\end{equation*}

## Usefulness of these metrics:

### Payback_Period : 

This tells us how fast we can recover investments from our property.
This may be one of the conservative estimates to evaluate expected profits from a property.



### Roi_equity : 

This metric is a good metric to assess profitability from a property considering only appreciation in price of property.
Because of variable rates of appreciation for each zipcode this metric gives a good estimate how location of zipcode determines ROI of a property 


### Roi : 

This metric would be useful to assess overall value of investment keeping in mind the increase in value of asset and also opens the value analysis  in case rental Property management decides to sell the current asset after 10 years. 
It is expected that Roi_without would be very similar in results in ROI_10  as it has high equity appreciation has same rate of increase as rate of rentals but ROI_10 . But some interesting insights can be drawn from differences in behavior of these two metrics for zipcodes.





**This is the file in which we create the required the dataframes as pickles for a feed to our visualzation as well as to do quality checks on the data.**


# Key Points:

1) select columns of interests in our analysis

2) Handling of missing values in the Data and doing necessary imputations.

3) Subsetting the data further as per needs for visualization

# Methodology:

## Airbnb Data:
The biggest concern for our analysis was imputations for missing zipcodes.Once we have data for zipcodes the city and state can easily be inferred from zipcode value.
We used pygeocoder a library built on google maps api to impute missing zipcodes by using latitudes and longitudes.
Luckily no missing values on lat and long.
After imputation only 2 properties still had missing values as their coordinates were not too specific for api to give a zipcode.
They were ignored and removed.



Once we imputed zipcode a quality check was done on zipcodes to ensure proper string length and then a range of zipcodes between 10000 and 20000 were selected as properties in NY.

Next we noticed 22 missing values for bedrooms which were removed from the dataset as their was no way to estimate the number of bedrooms.

finally we subset the 2 bedrooms properties in NY CITY.


This data set was then aggregated to be used for decision point 1 .

**
Note: the function impute zip will not work as api credential has been removed.The code will still work as the results from the imputation have been pickled.**











In [1]:
##### Reading datasets

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

zillow_data=pd.read_csv("Zip_Zhvi_2bedroom.csv")

#airbnb_data=pd.read_csv("listings.csv")
# airbnb_data.shape
# airbnb=airbnb_data[["id","city","state","zipcode","room_type","bedrooms","latitude","longitude","price","availability_365"]]
#airbnb.isnull().sum()

In [3]:
# from pygeocoder import Geocoder
# import pandas as pd
# import numpy as np
# from preprocessing import impute_zip
# # data=impute_zip(airbnb)
# data.to_pickle("final_frame.pkl")
airbnb=pd.read_pickle("pickles/final_frame.pkl")

In [4]:
airbnb.isnull().sum()

id                   0
city                62
state                6
zipcode              2
room_type            0
bedrooms            22
latitude             0
longitude            0
price                0
availability_365     0
dtype: int64

In [5]:
airbnb["zipcode"].isnull().sum()

2

In [6]:
airbnb[(airbnb.zipcode.str.len() > 5)|(airbnb.zipcode.str.len() < 5)]

,id,city,state,zipcode,room_type,bedrooms,latitude,longitude,price,availability_365
4062,2582812,Brooklyn,NY,11249\n11249,Private room,1.0,40.71997,-73.95945,$60.00,0
9112,6973292,New York,NY,11103-3233,Private room,1.0,40.76583,-73.91086,$59.00,0
10057,7724136,New York,NY,10002-2289,Private room,1.0,40.72092,-73.98849,$85.00,0
18482,14551508,Ridgewood,NY,11385-2308,Private room,1.0,40.70645,-73.90578,$45.00,0
26300,20968166,Springfield Gardens,NY,11413-3220,Entire home/apt,1.0,40.66648,-73.76286,$90.00,281
31255,24278511,Arverne,NY,11692-2304,Private room,1.0,40.58906,-73.79503,$95.00,364


In [7]:
airbnb.shape

(48895, 10)

In [8]:
airbnb[airbnb["zipcode"].isnull()]

,id,city,state,zipcode,room_type,bedrooms,latitude,longitude,price,availability_365
2736,1476238,New York,NY,None,Entire home/apt,1.0,40.76084,-73.96908,$300.00,98
4743,3351584,New York,NY,None,Private room,1.0,40.83473,-73.93706,$80.00,351


In [9]:
from preprocessing import zipcode_check
airbnb=zipcode_check(airbnb)
airbnb.zipcode.isnull().sum()

0

In [10]:
airbnb["zipcode"]=airbnb["zipcode"].astype(int)
airbnb_ny=airbnb[(airbnb["zipcode"]>=10000)&(airbnb["zipcode"]<=20000)].reset_index(drop=True)

In [11]:
airbnb.shape

(48887, 10)

In [12]:
airbnb.isnull().sum()

id                   0
city                62
state                6
zipcode              0
room_type            0
bedrooms            22
latitude             0
longitude            0
price                0
availability_365     0
dtype: int64

In [13]:
airbnb[airbnb["bedrooms"].isnull()].head()

,id,city,state,zipcode,room_type,bedrooms,latitude,longitude,price,availability_365
4,5022,New York,NY,10029,Entire home/apt,NaN,40.79851,-73.94399,$80.00,0
15,6848,Brooklyn,NY,11211,Entire home/apt,NaN,40.70837,-73.95352,$140.00,46
20,7801,Brooklyn,NY,11211,Entire home/apt,NaN,40.71842,-73.95718,$299.00,0
27,9357,New York,NY,10019,Entire home/apt,NaN,40.76715,-73.98533,$150.00,75
123,27883,New York,NY,10009,Entire home/apt,NaN,40.72245,-73.98527,$100.00,0


In [14]:
airbnb_ny=airbnb_ny.loc[airbnb_ny["bedrooms"]==2].reset_index(drop=True)
airbnb_ny.to_pickle("pickles/airbnb_ny.pkl")
airbnb_ny.isnull().sum()

id                  0
city                9
state               1
zipcode             0
room_type           0
bedrooms            0
latitude            0
longitude           0
price               0
availability_365    0
dtype: int64

## Zillow Data :

This dataset was filtered for properties in NY as no missing values were present on city column.
Then we selected the columns to be used for analysis.

We  took the column 2016-06 as current selling price of the property.
In this filtered data set we did not find any missing values.

However missing values were present in certain price years but we did not use them in our analysis.
While calculating rate, the fathest two columns were considered which had complete data in their columns.


**
Note: the function impute zip will not work as api credential has been removed.The code will still work as the results from the imputation have been pickled.**







In [15]:
zillow_data.City.isnull().sum()

0

In [16]:
## subsetting for city
from preprocessing import citysubset
zillow_data_ny=citysubset(zillow_data,"New York")
#zillow_data_ny.head()

## calculating rate

In [17]:
from metrics import rate
zillow_data_ny["rate"]=rate(zillow_data_ny,10)

In [18]:
data_zill=zillow_data_ny.iloc[:,0:7]
data_zill["rate"]=zillow_data_ny["rate"]
data_zill["2017-06"]=zillow_data_ny["2017-06"]

In [19]:
data_zill.isnull().sum()

RegionID      0
RegionName    0
City          0
State         0
Metro         0
CountyName    0
SizeRank      0
rate          0
2017-06       0
dtype: int64

In [20]:
data_zill["RegionName"]=data_zill["RegionName"].astype("str")
#data_zill=data_zill[data_zill["City"]=="New York"].reset_index(drop=True)
data_zill.rename(columns={"2017-06":"sell_price"},inplace=True)

## Finding County for each zipcode in airbnb_ny. Since airbnb has more rows than zillow data on individual property it was decided to use seperate counties files to merge with airbnb data

In [21]:
county=pd.read_csv('https://query.data.world/s/buotxun7n3q6twzd6havwil7h5nqpb')
data1=airbnb_ny.merge(county,how="left",left_on="zipcode",right_on="ZIP")
data1["price"]=pd.to_numeric(data1["price"].str.replace("[^\d\.]",""))
data1.loc[data1["room_type"]=="Private room","price"]=2*data1.loc[data1["room_type"]=="Private room","price"]

data1_county=data1.groupby(["COUNTYNAME","room_type"],as_index=False).mean()
data1_county.to_pickle("pickles/airbnb_county.pkl")
data2=data1.groupby(["COUNTYNAME","zipcode","room_type"],as_index=False).mean()
data2.to_pickle("pickles/airbnb_zip_county.pkl")

## Merging airbnb and zillow data

In [22]:
data1["zipcode"]=data1["zipcode"].astype(str)
air_zill=data1.reset_index(drop=True).merge(data_zill,how="inner",left_on="zipcode",right_on="RegionName")

## Calculating occupancy and payback rate

In [23]:
air_zill["Occupancy"]=0.75*365
air_zill["annual_rent"]=air_zill["price"]*air_zill["Occupancy"]
air_zill["payback_period"]=air_zill["sell_price"]/air_zill["annual_rent"]

## Calculating Roi's

In [24]:
from metrics import roi_equity,roi
air_zill["roi"]=roi(air_zill,10)
air_zill["roi_equity"]=roi_equity(air_zill,10)

In [25]:
data3=air_zill.groupby(["zipcode","CountyName"],as_index=False).mean()
data3.to_pickle("pickles/air_zillgroup.pkl")